In [ ]:
import tweepy
import time
from datetime import date

# Declaring API keys and Consumer Keys
# Generate your keys and replace with below placeholders
consumer_key = "C1fYDFlthopIA2gJclwxGpQL9"
consumer_secret = "AOUncdErnCGjUH3pbIwKjGVaLMo3Yawqnt3XzykxmNZ5VjaE2V"
access_token = "1386034465818902528-ZbVOXnCLTTZ73FQkRcnuvpwAFxnlId"
access_token_secret = "ah8QFti6hOKQFQnwlA3hc4TiwnftS8DHjtxi3D8luGj1S"

# Take the keys and authenticate the bot
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Establish an API object for using the methods
# Since the scale of tweets in this case is huge,
# the API should not stop when the rate limits are reached.
# Therefore, => wait_on_rate_limit is set to True
api = tweepy.API(auth, wait_on_rate_limit=True) 


# The bot searches for tweets with following hashtags
hashtags = [
    "#Remdesvir",
    "#remdesivir",
    "#ResourcesCOVID",
    "#oxygen","#Covishield_vaccine",
    "#covid",
    "#PlasmaRequirment",
    "#PlasmaDonor",
    "#BLOOD_DONATE",
    "#OxygenEmergency",
    "#DONATE_BLOOD",
    "#donations","#oxygen","#Oxygen"
   ]

# Hashtag for enabling already existing tweet threads
REPLY_HASHTAG = hashtags[2].lower()

# The bot will only look for tweets published on the present day
# 25 tweets at a time to avoid reaching rate limits
date_since = date.today()
tweet_count = 25

# Count number of fetching rounds done. Only for debug purpose.
polling_counter = 0

# Define a delay between each retweet to avoid processing overloading
RETWEET_DELAY = 1 #seconds

# Define the delay between each polling round to avoid
POLLING_DELAY = 1 #seconds

# Handle exceptions
def handle_exception(exp, tweet_id):
    error_code = str(exp.reason[10:13])
    if error_code == "327":
        print("  D ---Skipping: " + tweet_id)
    elif error_code == "139":
        print("  D ---Already liked: " + tweet_id)
    elif error_code == "185":
        print("  D ---!!!Rate limit reached")
    else:
        print("ERROR: ---" + exp.reason)
    return

# Retweet given a tweet's ID
def retweet(id):
    try:
        print("  T ---Retweeting: " + str(id))
        api.retweet(id)
        time.sleep(RETWEET_DELAY)

    except tweepy.TweepError as e:
        handle_exception(exp=e, tweet_id=str(id))

# Handle the #covidresourcebot special hashtag
def retweet_parent(tweet):

    # If tweet has a parent tweet, retweet parent
    if tweet.in_reply_to_status_id != "None":
        # try-catch avoid the bot from exiting the program
        try:
            print("  P ---Retweeting: " + str(tweet.in_reply_to_status_id))
            api.retweet(tweet.in_reply_to_status_id)
            time.sleep(RETWEET_DELAY)
        except tweepy.TweepError as e:
            handle_exception(exp=e, tweet_id=str(tweet.id))

    # else retweet the tweet itself
    else:
        retweet(tweet.in_reply_to_status_id)

# Define a search function that fetches and processes the tweets
def searchBot():
    for hashtag in hashtags:
        tweets = tweepy.Cursor(api.search, hashtag, since=date_since).items(tweet_count)
        for tweet in tweets:
            if REPLY_HASHTAG in tweet.text.lower():
                retweet_parent(tweet)
            else:
                retweet(tweet.id)
                try:
                    api.create_favorite(tweet.id)
                except tweepy.TweepError as e:
                    handle_exception(exp=e, tweet_id=str(tweet.id))

# Infinitely run the function with a polling delay
print("---------Starting the bot---------")
while(True):
    print("\n  D ---Polling Round: " + str(polling_counter))
    searchBot()
    time.sleep(POLLING_DELAY)
    polling_counter = polling_counter + 1

---------Starting the bot---------

  D ---Polling Round: 0
  T ---Retweeting: 1388420213582942210
  T ---Retweeting: 1388409390596706305
  T ---Retweeting: 1388409063172546563
  T ---Retweeting: 1388408259002847238
  T ---Retweeting: 1388406148903051266
  T ---Retweeting: 1388392531226857475
  T ---Retweeting: 1388388943163387906
  T ---Retweeting: 1388386500270776327
  T ---Retweeting: 1388382984609308673
  T ---Retweeting: 1388382745319989251
  D ---Skipping: 1388382745319989251
  D ---Already liked: 1388382745319989251
  T ---Retweeting: 1388381196745203715
  T ---Retweeting: 1388376958392692738
  D ---Skipping: 1388376958392692738
  D ---Already liked: 1388376958392692738
  T ---Retweeting: 1388376285622136834
  D ---Skipping: 1388376285622136834
  D ---Already liked: 1388376285622136834
  T ---Retweeting: 1388375228544258052
  T ---Retweeting: 1388375149078990850
  D ---Skipping: 1388375149078990850
  D ---Already liked: 1388375149078990850
  T ---Retweeting: 1388370509167939584


KeyboardInterrupt: ignored